In [1]:
"""
Simple VIIRS access
"""

import sys, os, shutil
import earthaccess
import geopandas as gpd
import pandas as pd
import rioxarray as rxr
import rasterio as rio
import math
import traceback
import datetime as dt
import xarray as xr
import pyproj
import datetime

from netCDF4 import Dataset 
from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
from affine import Affine
from osgeo import gdal, gdal_array, gdalconst, osr
from rasterio.transform import from_bounds
from scipy.spatial import cKDTree
from urllib.parse import urlparse

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
import logging
logging.getLogger('earthaccess').setLevel(logging.ERROR)

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *
        
# Directories
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

# Output directories
dataraw = os.path.join(projdir,'data/spatial/raw/VIIRS/')
datamod = os.path.join(projdir,'data/spatial/mod/VIIRS/')

# load the lookup table for pixel sizes
lut = pd.read_csv(os.path.join(projdir,'data/tabular/raw/pix_size_lut.csv'))

print("Ready !")

Ready !


In [2]:
def create_fire_gdf(extent, query):
    """ Creates a geodataframe with active fire detections from a directory with NetCDF files """

    granule_dfs = [] # to store the geolocated AFDs
    # granule_log = os.path.join(datamod, 'logs/processed_granules.txt')
    
    # Identify VNP14 vs. VNP03
    vnp14_files = [g.data_links()[0] for g in query if 'VNP14IMG' in g.data_links()[0]]
    vnp03_files = [g.data_links()[0] for g in query if 'VNP03IMG' in g.data_links()[0]]
    
    nprint = 10 # print counter
    for idx, vnp14 in enumerate(sorted(vnp14_files)):

        df = pd.DataFrame() # to store the active fire data
            
        # check if the granule has been processed
        url = urlparse(vnp14)
        granule_id = os.path.basename(url.path)    

        # gather information from file name
        timestamp = granule_id.split('.')[1:3]
        year = timestamp[0][1:5]
        day = timestamp[0][5:8]
        time = timestamp[1]
        date = dt.datetime.strptime(year+day, '%Y%j').strftime('%b %d') 
        acq_date = dt.datetime.strptime(year+day, '%Y%j').strftime('%-m/%-d/%Y')
        daytime = int(time) > 1500 #timestamps in the 1900h-2200h UTC range are afternoon for Western US
        
        # Identify the corresponding geolocation file
        geo_id = 'VNP03IMG.' + ".".join(timestamp)
        
        # Filter the search query to the matching VNP14 and VNP03
        query_ = [item for item in query if ".".join(timestamp) in item.data_links()[0]]
        # Open the VNP14IMG and gather the data
        fileset = earthaccess.open(query_)  
        vnp14 = fileset[1]
        vnp03 = fileset[0]
        
        with xr.open_dataset(vnp14, phony_dims='access') as vnp14ds:

            # Check for fire pixels in the specified region
            lonfp = vnp14ds.variables['FP_longitude'][:] # fire pixel longitude
            latfp = vnp14ds.variables['FP_latitude'][:] # fire pixel latitude
            fire_scene = ((lonfp > extent[0]) & (lonfp < extent[1]) & 
                          (latfp > extent[2]) & (latfp < extent[3]))
            if not fire_scene.any():  # Check for any fire pixels in region
                print(f"\n\tNo active fires detected in {granule_id}. Skipping...")
                # with open(granule_log, 'a') as log_file:
                #     log_file.write(f"{granule_id}\n") # log this granule as "processed"
                continue # skip if no fire pixels in region

            # granule attributes
            daynight = vnp14ds.DayNightFlag #string Day or Night
            granule_id = vnp14ds.LocalGranuleID

            # variables
            fire = vnp14ds['fire mask'] # the fire mask
            frp = vnp14ds.variables['FP_power'][:] # fire radiative power
            t4 = vnp14ds.variables['FP_T4'][:] # I04 brightness temp (kelvins)
            t5 = vnp14ds.variables['FP_T5'][:] # I05 brightness temp (kelvins)
            
            tree = cKDTree(np.array([lonfp, latfp]).T) #search tree for finding nearest FRP

            del fire_scene
            
        # Read the geolocation data 
        with xr.open_dataset(vnp03, group='geolocation_data', phony_dims='access') as geo_ds:
            i, j = np.indices(geo_ds.longitude.shape) #line and sample
            # Crop to fire bounding extent
            geo_scene = ((geo_ds.longitude > extent[0]) & (geo_ds.longitude < extent[1]) & 
                         (geo_ds.latitude > extent[2]) & (geo_ds.latitude < extent[3])).values
        
        # Populate the dataframe
        df['longitude'] = list(geo_ds.longitude.values[geo_scene])
        df['latitude'] = list(geo_ds.latitude.values[geo_scene])
        df['fire_mask'] = list(fire.values[geo_scene])
        df['confidence'] = pd.Categorical( df.fire_mask)
        df.confidence = df.confidence.replace(
            {0:'x', 1:'x', 2:'x', 3:'x', 4:'x', 5:'x', 6:'x', 7:'l', 8:'n', 9:'h'})
        df['daynight'] = daynight
        df['acq_date'] = acq_date
        df['acq_time'] = time
        df['granule_id'] = granule_id
        df['geo_id'] = geo_id
        df['j'] = list(j[geo_scene]) #sample number for pixel size lookup
        
        # Retain only low-high confidence fire points
        df = df[df.fire_mask>6] # keep only low-high confidence fire pixels
    
        # gather frp, brightness temps for nearest geolocated obs.
        for k in df.index:
            dist, nearest = tree.query([ df.loc[k, 'longitude'], df.loc[k, 'latitude'] ])
            df.loc[k, 'frp'] = frp[nearest].item()
            df.loc[k, 'iot4'] = t4[nearest].item()
            df.loc[k, 'iot5'] = t5[nearest].item()
    
        # Join to pixel size info
        df_ = pd.merge(df, lut, left_on='j', right_on='sample', how='left')
        df_.drop(columns=['j'], inplace=True)
        
        granule_dfs.append(df_) # append the granule dataframe

        # # write out the csv file
        # out_dir = os.path.join(datamod,'granules')
        # if not os.path.exists(out_dir):
        #     os.makedirs(out_dir)
        # out_fp = os.path.join(out_dir,f'{granule_id.replace(".","_")}.csv')
        # df_.to_csv(out_fp)

        # # write the granule id to the log file
        # with open(granule_log, 'a') as log_file:
        #     log_file.write(f"{granule_id}\n")

        # clear up some memory and log the processed granule
        # del df, i, j, geo_scene, fire, latfp, lonfp, frp, tree, df_
        # if self.download is True:
        #     os.remove(vnp14)
        #     os.remove(vnp03)
            
        if idx % nprint == 0:
            print(f"\n\tProcessed {idx+1} granules.\n")

    gc.collect() # clear out garbage
    
    # Concatenate the out dfs
    if len(granule_dfs) > 0:
        fire_data = pd.concat(granule_dfs) # for the entire list of granules
        return fire_data
    else:
        return None

In [28]:
# Load the fire dataset for the Southern Rockies
fp = os.path.join(projdir,'data/spatial/mod/NIFC/nifc-ics_2018_to_2023-aspen.gpkg')
fires = gpd.read_file(fp)

# subset to Southern Rockies
fires = fires[fires['na_l3name'] == 'Southern Rockies']

# tidy the fire id and name columns
fires.rename(columns={'NIFC_ID': 'Fire_ID', 'NIFC_NAME': 'Fire_Name'}, inplace=True)
# tify the date columns
fires['DISCOVERY_DATE'] = pd.to_datetime(fires['DISCOVERY_DATE']).dt.date
fires['WF_CESSATION_DATE'] = pd.to_datetime(fires['WF_CESSATION_DATE']).dt.date

# Adjust the start and end dates
fires['start_date'] = fires['DISCOVERY_DATE'] - timedelta(days=3)
fires['end_date'] = fires['WF_CESSATION_DATE'] + timedelta(days=3)

print(f"Available attributes: \n{fires.columns}")
print(f"\nThere are [{len(fires)}] fires.")

Available attributes: 
Index(['Fire_ID', 'Fire_Name', 'NIFC_ACRES', 'FINAL_ACRES', 'pct_aspen',
       'INCIDENT_ID', 'INCIDENT_NAME', 'START_YEAR', 'CAUSE', 'DISCOVERY_DATE',
       'DISCOVERY_DOY', 'WF_CESSATION_DATE', 'WF_CESSATION_DOY',
       'STR_DESTROYED_TOTAL', 'STR_DAMAGED_TOTAL', 'STR_THREATENED_MAX',
       'EVACUATION_REPORTED', 'PEAK_EVACUATIONS', 'WF_PEAK_AERIAL',
       'WF_PEAK_PERSONNEL', 'na_l3name', 'geometry', 'start_date', 'end_date'],
      dtype='object')

There are [67] fires.


In [ ]:
fire_names = ['CALF CANYON', 'SPRING CREEK']

In [4]:
fire = fires[fires['Fire_Name'] == fire_name]
fire.head()

,Fire_ID,Fire_Name,NIFC_ACRES,FINAL_ACRES,pct_aspen,INCIDENT_ID,INCIDENT_NAME,START_YEAR,CAUSE,DISCOVERY_DATE,...,WF_CESSATION_DOY,STR_DESTROYED_TOTAL,STR_DAMAGED_TOTAL,STR_THREATENED_MAX,EVACUATION_REPORTED,PEAK_EVACUATIONS,WF_PEAK_AERIAL,WF_PEAK_PERSONNEL,na_l3name,geometry
49,13356,CALF CANYON,299792.6,341735,3.322412,2022_14425222_HERMITS PEAK,HERMITS PEAK,2022,H,2022-04-06,...,165,903,903,22289,1,11973,33,1248.5,Southern Rockies,"MULTIPOLYGON (((-835981.980 1484287.075, -8359..."


In [5]:
print(fire['DISCOVERY_DATE'])
print(fire['WF_CESSATION_DATE'])

49    2022-04-06
Name: DISCOVERY_DATE, dtype: object
49    2022-06-14
Name: WF_CESSATION_DATE, dtype: object


In [6]:
coords, extent = get_coords(fire, 1000)

In [8]:
start_date = '2022-6-17'
end_date = '2022-6-18'

In [9]:
query = earthaccess.search_data(
    short_name=['VNP14IMG','VNP03IMG'], 
    polygon=coords,
    temporal=(str(start_date), str(end_date)), 
    cloud_hosted=True,
    count=-1
)

Granules found: 4


In [10]:
fire_df = create_fire_gdf(extent, query)

Opening 2 granules, approx size: 0.17 GB


QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]


	Processed 1 granules.

Opening 2 granules, approx size: 0.18 GB


QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]


	No active fires detected in VNP14IMG.A2022168.1954.002.2024076142025.nc. Skipping...


In [18]:
afds = fire_df
afds.head()

,longitude,latitude,fire_mask,confidence,daynight,acq_date,acq_time,granule_id,geo_id,frp,iot4,iot5,sample,along_scan,along_track,scan_angle,pix_area,geometry
0,-105.542000,35.969570,8,n,Night,6/17/2022,0830,VNP14IMG.A2022168.0830.002.2024076142027.nc,VNP03IMG.A2022168.0830,3.070913,317.730438,279.619446,2213,0.503154,0.408823,26.3315,0.205701,POINT (-105.54199981689453 35.96957015991211)
1,-105.543083,35.965866,8,n,Night,6/17/2022,0830,VNP14IMG.A2022168.0830.002.2024076142027.nc,VNP03IMG.A2022168.0830,3.025143,308.914368,279.967804,2213,0.503154,0.408823,26.3315,0.205701,POINT (-105.54308319091797 35.96586608886719)
2,-105.537354,35.965199,8,n,Night,6/17/2022,0830,VNP14IMG.A2022168.0830.002.2024076142027.nc,VNP03IMG.A2022168.0830,2.548720,302.209991,279.940247,2214,0.502861,0.408713,26.3048,0.205526,POINT (-105.537353515625 35.9651985168457)
3,-105.544876,35.958420,8,n,Night,6/17/2022,0830,VNP14IMG.A2022168.0830.002.2024076142027.nc,VNP03IMG.A2022168.0830,1.141477,296.525055,279.342712,2213,0.503154,0.408823,26.3315,0.205701,POINT (-105.54487609863281 35.95841979980469)
4,-105.545662,35.954678,8,n,Night,6/17/2022,0830,VNP14IMG.A2022168.0830.002.2024076142027.nc,VNP03IMG.A2022168.0830,1.141477,303.406616,280.043549,2213,0.503154,0.408823,26.3315,0.205701,POINT (-105.54566192626953 35.95467758178711)


In [19]:
afds.columns

Index(['longitude', 'latitude', 'fire_mask', 'confidence', 'daynight',
       'acq_date', 'acq_time', 'granule_id', 'geo_id', 'frp', 'iot4', 'iot5',
       'sample', 'along_scan', 'along_track', 'scan_angle', 'pix_area',
       'geometry'],
      dtype='object')

In [23]:
afds['confidence'] = afds['confidence'].astype(str)

In [24]:
from shapely.geometry import Point

# convert to spatial points using pixel centroid
afds['geometry'] = [Point(xy) for xy in zip(afds.longitude, afds.latitude)]
pix_gdf = gpd.GeoDataFrame(afds, geometry='geometry', crs="EPSG:4326")

# Reproject to a projected CRS for accurate buffering
pix_gdf = pix_gdf.to_crs("EPSG:5070")

# save this file out.
out_fp = os.path.join(datamod,'vnp14img_geo_CALF_CANYON_2.gpkg')
pix_gdf.to_file(out_fp)


pix_gdf.head() # check the results

,longitude,latitude,fire_mask,confidence,daynight,acq_date,acq_time,granule_id,geo_id,frp,iot4,iot5,sample,along_scan,along_track,scan_angle,pix_area,geometry
0,-105.542000,35.969570,8,n,Night,6/17/2022,0830,VNP14IMG.A2022168.0830.002.2024076142027.nc,VNP03IMG.A2022168.0830,3.070913,317.730438,279.619446,2213,0.503154,0.408823,26.3315,0.205701,POINT (-851385.523 1478108.680)
1,-105.543083,35.965866,8,n,Night,6/17/2022,0830,VNP14IMG.A2022168.0830.002.2024076142027.nc,VNP03IMG.A2022168.0830,3.025143,308.914368,279.967804,2213,0.503154,0.408823,26.3315,0.205701,POINT (-851523.443 1477705.693)
2,-105.537354,35.965199,8,n,Night,6/17/2022,0830,VNP14IMG.A2022168.0830.002.2024076142027.nc,VNP03IMG.A2022168.0830,2.548720,302.209991,279.940247,2214,0.502861,0.408713,26.3048,0.205526,POINT (-851021.395 1477579.990)
3,-105.544876,35.958420,8,n,Night,6/17/2022,0830,VNP14IMG.A2022168.0830.002.2024076142027.nc,VNP03IMG.A2022168.0830,1.141477,296.525055,279.342712,2213,0.503154,0.408823,26.3315,0.205701,POINT (-851766.492 1476892.128)
4,-105.545662,35.954678,8,n,Night,6/17/2022,0830,VNP14IMG.A2022168.0830.002.2024076142027.nc,VNP03IMG.A2022168.0830,1.141477,303.406616,280.043549,2213,0.503154,0.408823,26.3315,0.205701,POINT (-851878.402 1476482.232)


In [26]:
fp = os.path.join(datamod,'vnp14img_geo_CALF_CANYON_2.gpkg')
addin = gpd.read_file(fp)
len(addin)

13